<a href="https://colab.research.google.com/github/Dare-Badejo-001/BioSteam-TEA-LCA/blob/main/biosteamtutorialone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install Biosteam
!pip install biosteam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.5/429.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.5 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.6/936.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 4.5 MB/s eta 0

In [ ]:
# import biosteam
import biosteam as bst
from biosteam import settings

bst.nbtutorial()

settings.set_thermo(['Water', 'Methanol'])

feed =bst.Stream(Water=50, Methanol=20)

feed.show()

Stream: s1
phase: 'l', T: 298.15 K, P: 101325 Pa
flow (kmol/hr): Water     50
                Methanol  20


In [ ]:
# set prices for performing TEA later
feed.price = 0.15 #usd/kg
feed.cost

231.24018

In [ ]:
### Process Settings
'''
This includes the price of
0 - price of feeds and products
1 - conditions of utilities
2 - cost index  These are to be set before the simulating a system
'''

settings.CEPCI = 603.1 # settings to year 2018
settings.electricity_price = 0.065
cooling_water = settings.get_cooling_agent('cooling_water')
chilled_water = settings.get_cooling_agent('chilled_water')
cooling_water.T = 302  # change the temperature of the cooling water

In [ ]:
settings.cooling_agents

[<UtilityAgent: cooling_water>,
 <UtilityAgent: chilled_water>,
 <UtilityAgent: chilled_brine>,
 <UtilityAgent: propane>]

In [ ]:
cooling_water.show()

UtilityAgent: cooling_water
heat_transfer_efficiency: 1.000
heat_transfer_price: 0 USD/kJ
regeneration_price: 0.000488 USD/kmol
T_limit: 325 K
phase: 'l'
T: 302 K
P: 101325 Pa
flow (kmol/hr): Water  1


In [ ]:
chilled_water.show()

UtilityAgent: chilled_water
heat_transfer_efficiency: 1.000
heat_transfer_price: 5e-06 USD/kJ
regeneration_price: 0 USD/kmol
T_limit: 300 K
phase: 'l'
T: 280.37 K
P: 101325 Pa
flow (kmol/hr): Water  1


In [ ]:
settings.heating_agents

[<UtilityAgent: low_pressure_steam>,
 <UtilityAgent: medium_pressure_steam>,
 <UtilityAgent: high_pressure_steam>]

In [ ]:
lps = settings.get_heating_agent('low_pressure_steam')
lps.show()

UtilityAgent: low_pressure_steam
heat_transfer_efficiency: 0.950
heat_transfer_price: 0 USD/kJ
regeneration_price: 0.238 USD/kmol
T_limit: None
phase: 'g'
T: 412.19 K
P: 344738 Pa
flow (kmol/hr): Water  1


In [ ]:
lps.regeneration_price = 0.20
lps.show()

UtilityAgent: low_pressure_steam
heat_transfer_efficiency: 0.950
heat_transfer_price: 0 USD/kJ
regeneration_price: 0.2 USD/kmol
T_limit: None
phase: 'g'
T: 412.19 K
P: 344738 Pa
flow (kmol/hr): Water  1


In [ ]:
# add ammonia as a refrigerant :

Ammonia = bst.Chemical('Ammonia') # load ammonia properties
P = 101325 * 1.2  # pressure should be atmospheric
T = Ammonia.Tsat(P) # temperature should be at the bubble point for latent cooling

ammonia = bst.UtilityAgent(
    'ammonia',
    Ammonia=1,
    P=P,
    T=T,
    phase ='l',
    thermo = bst.Thermo([Ammonia]),
    heat_transfer_price= 13.17e-6,
)

settings.cooling_agents.append(ammonia)

In [ ]:
## Finding design requirements and costs with Unit Object

# specify vapor fraction and isobaric conditions

F1 = bst.Flash('F1', ins=feed, V=0.1, P=101325)
F1.show() # Show the conditions without simulations

Flash: F1
ins...
[0] s1  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] s3  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [ ]:
# Run the simulation and show the values again
F1.simulate()
F1.show() # this should give values for the outlet streams and the liquids and gas separations clearly showing

Flash: F1
ins...
[0] s1  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2  
    phase: 'g', T: 352.84 K, P: 101325 Pa
    flow (kmol/hr): Water     2.59
                    Methanol  4.41
[1] s3  
    phase: 'l', T: 352.84 K, P: 101325 Pa
    flow (kmol/hr): Water     47.4
                    Methanol  15.6


In [ ]:
F1.results()

Flash                                               Units            F1
Low pressure steam  Duty                            kJ/hr      5.92e+05
                    Flow                          kmol/hr          15.3
                    Cost                           USD/hr          3.06
Design              Vessel type                              Horizontal
                    Length                             ft          6.44
                    Diameter                           ft             4
                    Weight                             lb      1.38e+03
                    Wall thickness                     in         0.312
                    Vessel material                        Carbon steel
Purchase cost       Horizontal pressure vessel        USD      1.12e+04
                    Platform and ladders              USD      3.21e+03
                    Heat exchanger - Double pipe      USD       4.3e+03
Total purchase cost                                   USD      1.87e+04
Utility cost                                       USD/hr          3.06

In [ ]:
## Solving recycle loops and process specifications with system obects

# we will design a simple recycle process consisting of a flash and a partical liquid recycle loop

recycle = bst.Stream('liquid_recycle')
feed = bst.Stream('feed', Methanol=100, Water=450)

# mixer to mix both recycle and feed stream
M1 = bst.Mixer('M1', ins=(recycle, feed))

# Flash drum to set downstream the Mixer
F1 = bst.Flash('F1',
               ins=M1-0,
               outs=('vapor_product', 'liquid'),
               V=0.1, P=101325)

# Splitter downstream the flash

S1 = bst.Splitter('S1',
                  ins=F1-1,
                  outs=(recycle, 'liquid_product'),
                  split=0.5
                  )

In [ ]:
bst.main_flowsheet.diagram()

In [ ]:
sys = bst.main_flowsheet.create_system('sys')
sys.show()

System: sys
ins...
[0] feed  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] liquid_product  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [ ]:
# simulate the system then show the result
sys.simulate()
sys.show()

System: sys
Highest convergence error among components in recycle
stream S1-0 after 1 loops:
- flow rate   1.58e-01 kmol/hr (0.22%)
- temperature 1.34e-02 K (0.0037%)
ins...
[0] feed  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product  
    phase: 'g', T: 359.27 K, P: 101325 Pa
    flow (kmol/hr): Water     53.4
                    Methanol  46.6
[1] liquid_product  
    phase: 'l', T: 359.27 K, P: 101325 Pa
    flow (kmol/hr): Water     397
                    Methanol  53.5


In [ ]:
sys.results()

System                           Units      sys
Low pressure steam       Duty    kJ/hr 6.75e+06
                         Flow  kmol/hr      175
                         Cost   USD/hr     34.9
Total purchase cost                USD 3.87e+04
Installed equipment cost           USD 1.15e+05
Utility cost                    USD/hr     34.9
Material cost                   USD/hr        0
Sales                           USD/hr        0

# Creating a Unit

In [ ]:
# All units are created by passing a name and both inlet and outlet streams

from biosteam import Unit, Stream, settings, main_flowsheet
import biosteam as bst
bst.nbtutorial() # light-mode html diagra,s amd filter warnings

settings.set_thermo(['Water'])
ins = Stream('in0')
outs = Stream('out0')
U1 = Unit('U1', ins, outs)
U1.show(data=False)
U1.diagram()

Unit: U1
ins...
[0] in0  
outs...
[0] out0  


In [ ]:
from biosteam import Mixer, Splitter

feed1 = bst.Stream('feed1')
M1 = Mixer('M1', outs='s1')
S1 = Splitter('S1', outs=('s2', 'product1'), split=0.5)

feed2 = bst.Stream('feed2')
M2 = Mixer('M2', outs='s3')
S2 = Splitter('S2', outs=('recycle', 'product2'), split=0.5)

bst.main_flowsheet.diagram()

In [ ]:
# in pipe-notations

(feed1, S2-0)-M1-S1
(feed2, S1-0)

bst.main_flowsheet.diagram()